In [0]:
from pyspark.sql.functions import * 
from pyspark.sql.types import *
import time

### **Data Reading**

In [0]:
df= spark.read.format("parquet")\
  .load("abfss://bronze@databricksadlsete.dfs.core.windows.net/customers")
display(df)

In [0]:
df = df.drop("_rescued_data")
display(df)

In [0]:
#email we need to split organization name into a new column as domain
df=df.withColumn("domain",split(col("email"),"@")[1])
df.display()

In [0]:
df.groupBy("domain").agg((count("customer_id")).alias("total_customers")).orderBy(desc("total_customers")).display()

In [0]:
#Just for analysis 
df_gmail=df.filter(col("domain")=="gmail.com")
display(df_gmail)
time.sleep(5)

df_hotmail=df.filter(col("domain")=="hotmail.com")
df_hotmail.display()
time.sleep(5)

df_yahoo=df.filter(col("domain")=="yahoo.com")
df_yahoo.display()
time.sleep(5)


In [0]:
#concat first and last name
df=df.withColumn("Full_Name",concat(col("first_name"),lit(" "),col("last_name")))
df=df.drop("first_name","last_name")
df.display()

In [0]:
df.write.mode("overwrite").format("delta").save("abfss://silver@databricksadlsete.dfs.core.windows.net/customers")

In [0]:
%sql
CREATE TABLE IF NOT EXISTS databricks_ete_cata.silver.Customers_Silver
USING DELTA
LOCATION 'abfss://silver@databricksadlsete.dfs.core.windows.net/customers'

In [0]:
#%sql
#select * from databricks_ete_cata.silver.customers_silver